In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [8]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [ ]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 47947.3633 Validation Accuracy: 0.097656
Epoch  1, Batch   2 -Loss: 41406.7969 Validation Accuracy: 0.101562
Epoch  1, Batch   3 -Loss: 34828.2578 Validation Accuracy: 0.101562
Epoch  1, Batch   4 -Loss: 36737.9102 Validation Accuracy: 0.132812
Epoch  1, Batch   5 -Loss: 31290.1152 Validation Accuracy: 0.152344
Epoch  1, Batch   6 -Loss: 23527.5801 Validation Accuracy: 0.164062
Epoch  1, Batch   7 -Loss: 25450.1836 Validation Accuracy: 0.187500
Epoch  1, Batch   8 -Loss: 21849.8320 Validation Accuracy: 0.183594
Epoch  1, Batch   9 -Loss: 19388.1113 Validation Accuracy: 0.187500
Epoch  1, Batch  10 -Loss: 18631.6797 Validation Accuracy: 0.199219
Epoch  1, Batch  11 -Loss: 18435.7598 Validation Accuracy: 0.222656
Epoch  1, Batch  12 -Loss: 18442.1367 Validation Accuracy: 0.214844
Epoch  1, Batch  13 -Loss: 18052.1328 Validation Accuracy: 0.238281
Epoch  1, Batch  14 -Loss: 14524.4551 Validation Accuracy: 0.250000
Epoch  1, Batch  15 -Loss: 14466.8262 Validation

Epoch  1, Batch 122 -Loss:  4164.6431 Validation Accuracy: 0.664062
Epoch  1, Batch 123 -Loss:  3807.1016 Validation Accuracy: 0.660156
Epoch  1, Batch 124 -Loss:  3452.0952 Validation Accuracy: 0.652344
Epoch  1, Batch 125 -Loss:  2352.7285 Validation Accuracy: 0.652344
Epoch  1, Batch 126 -Loss:  3125.1118 Validation Accuracy: 0.652344
Epoch  1, Batch 127 -Loss:  3241.5486 Validation Accuracy: 0.648438
Epoch  1, Batch 128 -Loss:  3431.5166 Validation Accuracy: 0.664062
Epoch  1, Batch 129 -Loss:  2674.3149 Validation Accuracy: 0.656250
Epoch  1, Batch 130 -Loss:  2603.0942 Validation Accuracy: 0.660156
Epoch  1, Batch 131 -Loss:  2788.4395 Validation Accuracy: 0.652344
Epoch  1, Batch 132 -Loss:  2932.5369 Validation Accuracy: 0.652344
Epoch  1, Batch 133 -Loss:  3121.7607 Validation Accuracy: 0.656250
Epoch  1, Batch 134 -Loss:  2906.9773 Validation Accuracy: 0.656250
Epoch  1, Batch 135 -Loss:  4327.5078 Validation Accuracy: 0.656250
Epoch  1, Batch 136 -Loss:  2841.0679 Validation

Epoch  1, Batch 243 -Loss:  2267.8071 Validation Accuracy: 0.703125
Epoch  1, Batch 244 -Loss:  2936.6953 Validation Accuracy: 0.703125
Epoch  1, Batch 245 -Loss:  2121.4553 Validation Accuracy: 0.703125
Epoch  1, Batch 246 -Loss:  1631.9575 Validation Accuracy: 0.710938
Epoch  1, Batch 247 -Loss:  2163.8506 Validation Accuracy: 0.699219
Epoch  1, Batch 248 -Loss:  2134.1013 Validation Accuracy: 0.699219
Epoch  1, Batch 249 -Loss:  2066.5383 Validation Accuracy: 0.699219
Epoch  1, Batch 250 -Loss:  3032.7676 Validation Accuracy: 0.714844
Epoch  1, Batch 251 -Loss:  2672.6162 Validation Accuracy: 0.710938
Epoch  1, Batch 252 -Loss:  1356.2534 Validation Accuracy: 0.707031
Epoch  1, Batch 253 -Loss:  1571.1228 Validation Accuracy: 0.707031
Epoch  1, Batch 254 -Loss:  2322.5105 Validation Accuracy: 0.703125
Epoch  1, Batch 255 -Loss:  1467.9197 Validation Accuracy: 0.707031
Epoch  1, Batch 256 -Loss:  2009.1917 Validation Accuracy: 0.703125
Epoch  1, Batch 257 -Loss:  1860.9746 Validation

Epoch  1, Batch 364 -Loss:  1338.2046 Validation Accuracy: 0.730469
Epoch  1, Batch 365 -Loss:  1484.3795 Validation Accuracy: 0.726562
Epoch  1, Batch 366 -Loss:  1705.9496 Validation Accuracy: 0.730469
Epoch  1, Batch 367 -Loss:  1314.4609 Validation Accuracy: 0.734375
Epoch  1, Batch 368 -Loss:  2353.2112 Validation Accuracy: 0.726562
Epoch  1, Batch 369 -Loss:  1527.2798 Validation Accuracy: 0.726562
Epoch  1, Batch 370 -Loss:  2115.4863 Validation Accuracy: 0.730469
Epoch  1, Batch 371 -Loss:  1760.8635 Validation Accuracy: 0.734375
Epoch  1, Batch 372 -Loss:  2059.5361 Validation Accuracy: 0.730469
Epoch  1, Batch 373 -Loss:  2782.0759 Validation Accuracy: 0.726562
Epoch  1, Batch 374 -Loss:  2104.2605 Validation Accuracy: 0.734375
Epoch  1, Batch 375 -Loss:  1425.8939 Validation Accuracy: 0.742188
Epoch  1, Batch 376 -Loss:  1932.8311 Validation Accuracy: 0.746094
Epoch  1, Batch 377 -Loss:  1733.1538 Validation Accuracy: 0.738281
Epoch  1, Batch 378 -Loss:  1393.9462 Validation

Epoch  2, Batch  56 -Loss:  1599.3279 Validation Accuracy: 0.753906
Epoch  2, Batch  57 -Loss:  1295.0192 Validation Accuracy: 0.761719
Epoch  2, Batch  58 -Loss:  1253.6162 Validation Accuracy: 0.769531
Epoch  2, Batch  59 -Loss:  1124.4186 Validation Accuracy: 0.769531
Epoch  2, Batch  60 -Loss:  1247.9889 Validation Accuracy: 0.769531
Epoch  2, Batch  61 -Loss:  1221.6506 Validation Accuracy: 0.765625
Epoch  2, Batch  62 -Loss:  1173.9351 Validation Accuracy: 0.765625
Epoch  2, Batch  63 -Loss:  1239.1414 Validation Accuracy: 0.765625
Epoch  2, Batch  64 -Loss:  1708.0459 Validation Accuracy: 0.765625
Epoch  2, Batch  65 -Loss:  1771.3748 Validation Accuracy: 0.769531
Epoch  2, Batch  66 -Loss:  1077.7502 Validation Accuracy: 0.765625
Epoch  2, Batch  67 -Loss:  1301.1350 Validation Accuracy: 0.769531
Epoch  2, Batch  68 -Loss:  1828.5363 Validation Accuracy: 0.765625
Epoch  2, Batch  69 -Loss:  1611.9722 Validation Accuracy: 0.761719
Epoch  2, Batch  70 -Loss:  1742.9982 Validation

Epoch  2, Batch 177 -Loss:  1296.8984 Validation Accuracy: 0.789062
Epoch  2, Batch 178 -Loss:  1691.0043 Validation Accuracy: 0.789062
Epoch  2, Batch 179 -Loss:  1189.0894 Validation Accuracy: 0.789062
Epoch  2, Batch 180 -Loss:  1553.0126 Validation Accuracy: 0.789062
Epoch  2, Batch 181 -Loss:  1012.5938 Validation Accuracy: 0.785156
Epoch  2, Batch 182 -Loss:   914.8912 Validation Accuracy: 0.785156
Epoch  2, Batch 183 -Loss:  1075.2185 Validation Accuracy: 0.792969
Epoch  2, Batch 184 -Loss:  1156.0377 Validation Accuracy: 0.792969
Epoch  2, Batch 185 -Loss:  1403.3564 Validation Accuracy: 0.789062
Epoch  2, Batch 186 -Loss:  1632.2357 Validation Accuracy: 0.792969
